# Heuristic Solution: Independent Segment Effects (ISE)
## Robust Configuration Selection for Multiple Workload Scenarios

### Imports

In [1]:
# import pyomo
from pyomo.environ import *
from pyomo.opt import SolverFactory

# import pandas
import pandas as pd

# import helper
from helper.print import print_result
from helper.export import export_config

### Load benchmark, system calibration and workload data

In [2]:
# performance measurements of the benchmark queries
df_perf = pd.read_csv('../data/benchmark/runtimes.csv')
# memory consumption of segments
df_memory = pd.read_csv('../data/benchmark/memory_consumption.csv')
# memory consumption of indexes
df_memory_index = pd.read_csv('../data/benchmark/memory_consumption_index.csv')

# calibration data for hyrise 
df_poslist_scan_penalty = pd.read_csv('../data/calibration/poslist_scan_penalty.csv')
# calibration data for storage devices
df_storage_penalty = pd.read_csv('../data/calibration/storage_penalty.csv')

# Apply indexes on pandas dataframes
df_perf.set_index(['ORDER_BY', 'ENCODING', 'SCAN_COLUMN', 'SELECTIVITY', 'INDEX', 'SCAN_TYPE'], inplace=True)
df_memory.set_index(['ORDER_BY', 'ENCODING', 'COLUMN', 'CHUNK_ID'], inplace=True)
df_memory_index.set_index(['ORDER_BY', 'ENCODING', 'COLUMN', 'CHUNK_ID'], inplace=True)
df_poslist_scan_penalty.set_index(['ENCODING'], inplace=True)
df_storage_penalty.set_index(['STORAGE', 'ENCODING', 'INDEX'], inplace=True)

### Workloads

In [3]:
WORKLOAD_PATH = '../data/workloads/'

def load_workload(folder, path=WORKLOAD_PATH):
    chunk_access = pd.read_csv(path + folder + '/chunk_access.csv')
    chunk_access.set_index(['QUERY_ID', 'CHUNK'], inplace=True)
    workload = pd.read_csv(path + folder + '/workload.csv')
    
    return workload, chunk_access

## Model

In [4]:
# In Hyrise, scan operations performed on the pos list are significantly slower than isolated 
# executed scan operations. To consider this in our cost estimation, we introduce an encoding-specific 
# parameter, which can be measured during the model calibration. 
def scan_order_penalty(scan_order, encoding):
    return 1 if (scan_order == 0) else df_poslist_scan_penalty.loc[(encoding)]['PENALTY']

# storage penalties
def storage_penalty(encoding, index, storage):
    return df_storage_penalty.loc[(storage, encoding, index)]['PENALTY']
    
# In the ISE model, we distinguish between scan operations on a sorted and unsorted segment.  
# To get the execution time of unsorted column scan (if o == 0) by setting the order_by_value 
# to the id of the unsorted scan else to the id of the scan column
def order_by_value(o, n):
    return UNSORTED_ID if o == 0 else n

# set index value to 0 for all scans with a scan order value >= 1
# indexes can only be used to speed up the first scan
def index_value(scan, i):    
    return i if scan['SCAN_ORDER'] == 0 else 0

# the measured execution time of an isolated scan operation with the given column configuration
def performance(e, o, i, scan):
    return df_perf.loc[(o, e, scan['SCAN_COLUMN'], scan['SELECTIVITY'], index_value(scan, i), scan['SCAN_TYPE'])]['TIME']

# returns the approximated costs of single segment scan based on the overall column scan costs
# proportional costs depending on the number of accessed chunks
def segment_access(m, n, scan, df_chunk_access):
    q = scan['QUERY_ID']
 
    if scan['SCAN_COLUMN'] != n or df_chunk_access.loc[(q, m)]['ACCESSED'] == 0:
        return 0
    return (1/df_chunk_access.loc[(q, slice(None))]['ACCESSED'].sum())
    
# updates the storage budgets (the number of storage units can not be modified)
def update_storage_budgets(storage_budgets, model):
    model.SB.reconstruct(storage_budgets)
    model.MemoryBudgetConstraint.reconstruct()

In [6]:
model = ConcreteModel()

# memory budgets for storage devices    
STORAGE_BUDGET = {0:3_000_000_000, 1:3_000_000_000}

# defined workloads with chunk accessed information
WORKLOAD = ['workload_1', 'workload_2', 'workload_3']

# defined in the benchmark that unsorted is the max value in the ORDER_BY column 
UNSORTED_ID = df_perf.index.levels[0].max()

# upper boundary that has to be minimized 
model.L = Var(within=NonNegativeReals)

# workoad index  
model.W = Set(initialize=range(0, len(WORKLOAD)))

# set of chunks
model.M = Set(initialize=df_memory.index.levels[3].unique())

# set of columns
model.N = Set(initialize=df_perf.index.levels[2].unique())

# set of encodings
model.E = Set(initialize=df_perf.index.levels[1].unique())

# set of storage devices
model.B = Set(initialize=range(0, len(STORAGE_BUDGET)))

# discrete variable for the sorting configuration of a segment
model.O = Set(initialize=[0,1], within=Binary)

# discrete variable for the indexing configuration of a segment
model.I = Set(initialize=[0,1], within=Binary)

# decision variable to describe the selected configuration option
model.X = Var(model.M, model.N, model.E, model.O, model.I, model.B, within=Binary)  

# storage budget for a given storage device 
model.SB = Param(model.B, within=NonNegativeIntegers, initialize=STORAGE_BUDGET, mutable=True)

# runtime performance for a segment per encoding, sorting, indexing, and selectivity 
def performance_init(model, m, n, e, o, i, b, w):
    df_workload, df_chunk_access = load_workload(WORKLOAD[w])
    scan_operations = range(0, df_workload.shape[0])
    
    return sum(df_workload.iloc[s]['FREQUENCY'] * \
               performance(e, o, i, df_workload.iloc[s]) * \
               segment_access(m, n, df_workload.iloc[s], df_chunk_access) * \
               df_workload.iloc[s]['SCAN_FACTOR'] * \
               scan_order_penalty(df_workload.iloc[s]['SCAN_ORDER'], e) * \
               storage_penalty(e, i, b)
               for s in scan_operations)
model.C = Param(model.M, model.N, model.E, model.O, model.I, model.B, model.W, within=NonNegativeReals, initialize=performance_init, mutable=True)

# memory consumption for a segment for a given encoding, sorting, and indexing configuration
# if an index is applied, the aggregated memory consumption is returned 
def memory_init(model, m, n, e, o, i):    
    if i == 0:
        return df_memory.loc[(order_by_value(o, n), e, n, m)]['SIZE_IN_BYTES']
    else:
        return df_memory.loc[(order_by_value(o, n), e, n, m)]['SIZE_IN_BYTES'] + \
               df_memory_index.loc[(order_by_value(o, n), e, n, m)]['SIZE_IN_BYTES']
model.MC = Param(model.M, model.N, model.E, model.O, model.I, within=NonNegativeReals, initialize=memory_init)

# returns if a given index encoding configuration can be implemented in a database
# for hyrise only indices on dictionary encoded columns are allowed
def valid_index_encoding_config(model, e, i):
    if i == 1 and e == 0:
        return 1
    return 0
model.VC = Param(model.E, model.I, within=Binary, initialize=valid_index_encoding_config)

### Objective

# upper boundary for the runtime for a workload (worst-case optimization)
model.Obj = Objective(expr=model.L)

### Model specific consraints

# objective to minimize the costs over all segments 
def runtime_rule(m, w):
    return sum(m.X[chunk_id, column_id, encoding_id, order_config, index_config, storage_id] * \
               m.C[chunk_id, column_id, encoding_id, order_config, index_config, storage_id, w]
               for chunk_id in m.M
               for column_id in m.N
               for encoding_id in m.E
               for order_config in m.O
               for index_config in m.I
               for storage_id in m.B) <= m.L
model.RuntimeConstraint = Constraint(model.W, rule=runtime_rule)

# Size within budget for each storage unit
def memory_budget_rule(m, b):
    # sum up memory consumption of all 
    return sum(m.X[chunk_id, column_id, encoding_id, order_config, index_config, b] * \
               m.MC[chunk_id, column_id, encoding_id, order_config, index_config]
               for chunk_id in m.M
               for column_id in m.N
               for encoding_id in m.E
               for order_config in m.O
               for index_config in m.I) <= m.SB[b]
model.MemoryBudgetConstraint = Constraint(model.B, rule=memory_budget_rule)

# one column sorted per chunk
def single_sorting_column_per_chunk_rule(m, i):
    return sum(m.X[i, column_id, encoding_id, 1, index_config, storage_id]
                for column_id in m.N
                for encoding_id in m.E
                for index_config in m.I
                for storage_id in m.B) <= 1
model.SingleSortColumnConstraints = Constraint(model.M, rule=single_sorting_column_per_chunk_rule)

# exactly one encoding, sorting, indexing, and storage configuration per segment
def one_encoding_sorting_indexing_config_active_per_segment_rule(m, i, j):
    return sum(m.X[i, j, encoding_id, order_config, index_config, storage_id] 
                for encoding_id in m.E
                for order_config in m.O
                for index_config in m.I
                for storage_id in m.B) == 1
model.SingleEncodingConstraints = Constraint(model.M, model.N, rule=one_encoding_sorting_indexing_config_active_per_segment_rule)

# all segmets of a chunk have to be stored on the same storage medium
def single_storage_medium_per_chunk_rule(m, i, j):
    return sum(m.X[i, 1, encoding_id, order_config, index_config, storage_id] * storage_id
               for encoding_id in m.E
               for order_config in m.O
               for index_config in m.I
               for storage_id in m.B) == \
           sum(m.X[i, j, encoding_id, order_config, index_config, storage_id] * storage_id
               for encoding_id in m.E
               for order_config in m.O
               for index_config in m.I
               for storage_id in m.B)       
model.SingleStorageConstraints = Constraint(model.M, model.N, rule=single_storage_medium_per_chunk_rule)

### Database (Hyrise) specific constraints 

# Indexed columns need to have dictionary encoding
def valid_index_configuration_rule(m, i, j, k, l, n):
    return m.X[i, j, k, l, 1, n] <= m.VC[k, 1]
model.IndexesOnDictColumnsConstraints = Constraint(model.M, model.N, model.E, model.O, model.B, rule=valid_index_configuration_rule)

# Solving

In [7]:
solver = SolverFactory('gurobi')
solver.options['threads'] = 16

result = solver.solve(model)
print_result(result, model, True)

Solving for budget;
  Storage: 0     Storage Size: 3000000000 
  Storage: 1     Storage Size: 3000000000 

Result: optimal (walltime: 0.0493s)
Objective: 109295123758.33327
Memory consumption:
  0: 2999.343622 MB
  1: 0.0 MB

Storage: 0
CHUNK               driver_id           latitude            longitude           timestamp           status              
0                   - - Dictionary      S - FoR-SIMD        - - Unencoded       - - Dictionary      - - LZ4             
1                   - - Dictionary      S - FoR-SIMD        - - FoR-SIMD        - - RunLength       - - FoR-SIMD        
2                   - - Dictionary      S - RunLength       - - Unencoded       - - RunLength       - - FoR-SIMD        
3                   - - Dictionary      S - RunLength       - - Unencoded       - - Dictionary      - - FoR-SIMD        
4                   - - Dictionary      S - RunLength       - - Unencoded       - - FoR-SIMD        - - FoR-SIMD        
5                   - - Dictionary   